In [15]:
import time
import re
import random
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
from openpyxl import load_workbook
from openpyxl.worksheet.datavalidation import DataValidation

# ============== CONFIGURATION ==============
# NOTE TO SELF: Change these values for different searches
# For parallel scraping in Jupyter: duplicate notebook, change START_PAGE/END_PAGE in each

SEARCH_TERM = "real-estate"            # e.g., "general-contractors", "real-estate-agents", "plumbers", "lawyers"
LOCATION = "queens-ny"                 # e.g., "brooklyn-ny", "queens-ny", "manhattan-ny"
INDUSTRY_LABEL = "Real Estate"         # Label for the Industry column in output

# === CHANGE THESE FOR PARALLEL RUNS ===
START_PAGE = 2                         # Page to start from
END_PAGE = 4                           # Page to end at (inclusive)
# Notebook 1: START=1, END=1
# Notebook 2: START=2, END=2  
# Notebook 3: START=3, END=3
# (One page per notebook to avoid session death)
# ======================================

OUTPUT_FILE = f"yp_{LOCATION}_{SEARCH_TERM}_p{START_PAGE}-{END_PAGE}.xlsx"

FETCH_EMAILS = True                    # Set False for faster scraping (no emails)
DEBUG = False                          
HEADLESS = False                       # False = visible browser (bypasses Cloudflare better)
MIN_DELAY = 5                          # Increased delay
MAX_DELAY = 10                         # Increased delay
PAGE_DELAY = 15                        # Seconds to wait between pages (longer to avoid detection)
RESTART_DRIVER_EACH_PAGE = True        # Restart browser between pages to get fresh session
# ===========================================


def create_driver():
    options = Options()
    
    if HEADLESS:
        options.add_argument("--headless=new")
    
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-gpu")
    options.add_argument("--window-size=1920,1080")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    return driver


def extract_email_from_website(driver, website_url):
    """Try to find email on company's own website"""
    if not website_url or website_url == "N/A":
        return ""
    
    try:
        # Clean up URL
        if not website_url.startswith("http"):
            website_url = "https://" + website_url
        
        time.sleep(random.uniform(1, 2))
        driver.get(website_url)
        time.sleep(2)
        
        page_source = driver.page_source
        
        # Check if page loaded (not error)
        if "404" in driver.title or "not found" in driver.title.lower():
            return ""
        
        # Method 1: Find mailto links
        mailto_match = re.search(r'href=["\']mailto:([^"\'<>?\s]+)', page_source, re.IGNORECASE)
        if mailto_match:
            email = mailto_match.group(1).strip()
            if '@' in email:
                return email
        
        # Method 2: Find email patterns in page
        # More specific pattern to avoid false positives
        email_matches = re.findall(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}', page_source)
        for email in email_matches:
            email_lower = email.lower()
            # Filter out common false positives
            if not any(x in email_lower for x in ['example.com', 'domain.com', 'email.com', 'yoursite', 
                                                    'sentry.io', 'schema.org', 'json', 'wixpress', 
                                                    'googleapis', 'facebook', 'twitter', '.png', '.jpg']):
                return email
        
        # Method 3: Try common contact page URLs
        base_url = website_url.rstrip('/')
        contact_pages = ['/contact', '/contact-us', '/about', '/about-us']
        
        for contact_path in contact_pages:
            try:
                driver.get(base_url + contact_path)
                time.sleep(1.5)
                contact_source = driver.page_source
                
                mailto_match = re.search(r'href=["\']mailto:([^"\'<>?\s]+)', contact_source, re.IGNORECASE)
                if mailto_match:
                    email = mailto_match.group(1).strip()
                    if '@' in email:
                        return email
                
                email_matches = re.findall(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}', contact_source)
                for email in email_matches:
                    email_lower = email.lower()
                    if not any(x in email_lower for x in ['example.com', 'domain.com', 'email.com', 'yoursite',
                                                           'sentry.io', 'schema.org', 'json', 'wixpress',
                                                           'googleapis', 'facebook', 'twitter', '.png', '.jpg']):
                        return email
            except:
                continue
                
    except Exception as e:
        pass
    
    return ""


def extract_email_from_detail(driver, detail_url, website_url="", debug_save=False):
    """Try Yellow Pages first, then fall back to company website"""
    try:
        # Random delay before request to appear more human
        time.sleep(random.uniform(MIN_DELAY, MAX_DELAY))
        
        driver.get(detail_url)
        
        # Wait for page to fully load
        try:
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, ".business-info, .sales-info, #main-content, #cf-wrapper"))
            )
        except:
            pass
        
        time.sleep(2)
        
        # Get page source
        page_source = driver.page_source
        
        # Check if we got Cloudflare blocked
        is_blocked = "you have been blocked" in page_source.lower() or ("cloudflare" in page_source.lower() and "ray id" in page_source.lower())
        
        if is_blocked:
            # Try company website instead
            if website_url:
                print(" [YP blocked, trying website]", end="")
                email = extract_email_from_website(driver, website_url)
                if email:
                    return email
            return "__BLOCKED__"
        
        # Debug: save page source to file
        if debug_save:
            with open("debug_page_source.html", "w", encoding="utf-8") as f:
                f.write(page_source)
            print(f" [DEBUG: saved]", end="")
        
        # Scroll down to load lazy content
        driver.execute_script("window.scrollTo(0, 800);")
        time.sleep(1)
        page_source = driver.page_source
        
        # Method 1: Direct regex on page source for mailto: links (most reliable)
        mailto_match = re.search(r'href=["\']mailto:([^"\'<>?\s]+)', page_source, re.IGNORECASE)
        if mailto_match:
            email = mailto_match.group(1).strip()
            if '@' in email and 'yellowpages' not in email.lower():
                return email
        
        # Method 2: Selenium - grab href from email-business link
        try:
            email_elements = driver.find_elements(By.CSS_SELECTOR, "a.email-business, a[class*='email']")
            for el in email_elements:
                href = el.get_attribute("href") or ""
                if "mailto:" in href:
                    return href.replace("mailto:", "").split("?")[0].strip()
        except:
            pass
        
        # Method 3: Selenium - any mailto link
        try:
            mailto_links = driver.find_elements(By.CSS_SELECTOR, "a[href*='mailto:']")
            for link in mailto_links:
                href = link.get_attribute("href") or ""
                if "mailto:" in href:
                    email = href.replace("mailto:", "").split("?")[0].strip()
                    if '@' in email:
                        return email
        except:
            pass
        
        # Method 4: BeautifulSoup parsing
        soup = BeautifulSoup(page_source, "html.parser")
        
        # Find all links and check href
        for link in soup.find_all("a", href=True):
            href = link["href"]
            if "mailto:" in href:
                email = href.replace("mailto:", "").split("?")[0].strip()
                if '@' in email and 'yellowpages' not in email.lower():
                    return email
        
        # Method 5: Find email pattern anywhere in page
        email_pattern = re.search(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}', page_source)
        if email_pattern:
            email = email_pattern.group()
            # Filter out false positives
            if not any(x in email.lower() for x in ['example.com', 'domain.com', 'yellowpages', 'schema.org', 'json']):
                return email
        
        # No email on YP page - try company website as last resort
        if website_url:
            print(" [trying website]", end="")
            email = extract_email_from_website(driver, website_url)
            if email:
                return email
            
    except Exception as e:
        print(f" [err: {e}]", end="")
    return ""


def parse_listing(listing):
    try:
        company = listing.select_one(".business-name span").text.strip()
        phone = listing.select_one(".phones").text.strip() if listing.select_one(".phones") else ""
        
        address_street = listing.select_one(".street-address")
        address_locality = listing.select_one(".locality")
        full_address = " ".join(filter(None, [
            address_street.text.strip() if address_street else "",
            address_locality.text.strip() if address_locality else ""
        ]))
        
        website_el = listing.select_one(".track-visit-website")
        website = website_el["href"] if website_el else ""
        
        detail_link = "https://www.yellowpages.com" + listing.select_one(".business-name")["href"]

        return {
            "#": None,
            "Company Name": company,
            "Industry": INDUSTRY_LABEL,
            "Contact Name": "",
            "Email Address": "",
            "Phone Number": phone,
            "Website URL": website,
            "Address": full_address,
            "Date Added": datetime.now().strftime("%-m/%-d/%y"),
            "Date Contacted": "",
            "Source": detail_link,
            "Notes": "",
            "Called": "",
            "Followed Up": "",
            "Closed": ""
        }
    except Exception as e:
        print(f"  Skipping listing: {e}")
        return None


def add_checkboxes(filepath):
    wb = load_workbook(filepath)
    ws = wb.active
    
    checkbox_validation = DataValidation(type="list", formula1='"☐,☑"', allow_blank=True)
    ws.add_data_validation(checkbox_validation)
    
    headers = {cell.value: cell.column for cell in ws[1]}
    
    for col_name in ["Called", "Followed Up", "Closed"]:
        if col_name in headers:
            col_idx = headers[col_name]
            for row in range(2, ws.max_row + 1):
                cell = ws.cell(row=row, column=col_idx)
                cell.value = "☐"
                checkbox_validation.add(cell)
    
    wb.save(filepath)


def get_listings_from_page(driver):
    """Extract all listing data from current page"""
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".result"))
        )
    except:
        return []
    
    listings = driver.find_elements(By.CSS_SELECTOR, ".result")
    page_data = []
    
    for listing in listings:
        try:
            soup = BeautifulSoup(listing.get_attribute("outerHTML"), "html.parser")
            parsed = parse_listing(soup)
            if parsed:
                page_data.append(parsed)
        except:
            continue
    
    return page_data


def scrape_yellowpages():
    base_url = f"https://www.yellowpages.com/{LOCATION}/{SEARCH_TERM}"
    print(f"=" * 60)
    print(f"Yellow Pages Scraper")
    print(f"=" * 60)
    print(f"Search: {SEARCH_TERM}")
    print(f"Location: {LOCATION}")
    print(f"Pages: {START_PAGE} to {END_PAGE}")
    print(f"Fetch emails: {FETCH_EMAILS}")
    print(f"Output: {OUTPUT_FILE}")
    print(f"=" * 60 + "\n")
    
    driver = create_driver()
    all_data = []
    
    try:
        for page in range(START_PAGE, END_PAGE + 1):
            try:
                # Build URL - page 1 has no param, others have ?page=N
                url = base_url if page == 1 else f"{base_url}?page={page}"
                
                print(f"[Page {page}] Loading {url}")
                driver.get(url)
                time.sleep(2)
                
                # Check if we got results
                page_data = get_listings_from_page(driver)
                
                if not page_data:
                    print(f"  No listings found - stopping pagination")
                    break
                
                print(f"  Found {len(page_data)} listings")
                
                # Fetch emails from detail pages
                if FETCH_EMAILS:
                    emails_found = 0
                    for i, entry in enumerate(page_data):
                        try:
                            company_short = entry['Company Name'][:35].ljust(35)
                            print(f"  [{i+1:2}/{len(page_data)}] {company_short}", end="", flush=True)
                            
                            debug_save = DEBUG and (page == START_PAGE and i == 0)
                            email = extract_email_from_detail(
                                driver, 
                                entry["Source"], 
                                website_url=entry.get("Website URL", ""),
                                debug_save=debug_save
                            )
                            
                            if email == "__BLOCKED__":
                                print(f" → (blocked, no website email found)")
                            elif email:
                                entry["Email Address"] = email
                                emails_found += 1
                                print(f" → {email}")
                            else:
                                print(f" → (no email)")
                        except Exception as e:
                            print(f" → [error: {e}]")
                            continue
                    
                    print(f"  Page {page} complete: {emails_found}/{len(page_data)} emails found\n")
                
                all_data.extend(page_data)
                
                # === SAVE PROGRESS AFTER EACH PAGE ===
                if all_data:
                    temp_df = pd.DataFrame(all_data)
                    for idx, row in enumerate(all_data, start=1):
                        row["#"] = idx
                    temp_df.to_excel(OUTPUT_FILE, index=False)
                    print(f"  💾 Progress saved to {OUTPUT_FILE} ({len(all_data)} leads)\n")
                
                # Random delay between pages + optionally restart driver
                if page < END_PAGE:
                    # Close and restart browser to get fresh session
                    if RESTART_DRIVER_EACH_PAGE:
                        print(f"  🔄 Restarting browser for fresh session...")
                        try:
                            driver.quit()
                        except:
                            pass
                        time.sleep(3)
                        driver = create_driver()
                    
                    delay = random.uniform(PAGE_DELAY, PAGE_DELAY + 5)
                    print(f"  ⏳ Waiting {delay:.1f}s before next page...\n")
                    time.sleep(delay)
                    
            except Exception as page_error:
                print(f"\n  ⚠️ Error on page {page}: {page_error}")
                print(f"  Saving progress and continuing...\n")
                # Try to restart driver
                try:
                    driver.quit()
                except:
                    pass
                driver = create_driver()
                continue
                
    except Exception as e:
        print(f"\n❌ Fatal error: {e}")
    finally:
        try:
            driver.quit()
        except:
            pass
    
    if not all_data:
        print("No data collected!")
        return pd.DataFrame()
    
    # Add row numbers
    for i, row in enumerate(all_data, start=1):
        row["#"] = i

    # Save to Excel
    df = pd.DataFrame(all_data)
    df.to_excel(OUTPUT_FILE, index=False)
    add_checkboxes(OUTPUT_FILE)
    
    # Summary
    emails_count = sum(1 for row in all_data if row["Email Address"])
    print(f"=" * 60)
    print(f"COMPLETE!")
    print(f"=" * 60)
    print(f"Total leads: {len(all_data)}")
    print(f"With emails: {emails_count}")
    print(f"Saved to: {OUTPUT_FILE}")
    print(f"=" * 60)
    
    return df


# === RUN SCRAPER (Cell 1) ===
df = scrape_yellowpages()


# === MERGE ALL FILES (Cell 2 - run after all notebooks finish) ===
# Put this in a separate cell, run once when all parallel scrapes are done:
#
# import pandas as pd
# import glob
# files = glob.glob("yp_*.xlsx")
# df = pd.concat([pd.read_excel(f) for f in files], ignore_index=True)
# df["#"] = range(1, len(df) + 1)
# df.drop_duplicates(subset=["Company Name", "Phone Number"], inplace=True)
# df.to_excel("yp_combined_leads.xlsx", index=False)
# print(f"Merged {len(files)} files → {len(df)} unique leads")

Yellow Pages Scraper
Search: real-estate
Location: queens-ny
Pages: 2 to 4
Fetch emails: True
Output: yp_queens-ny_real-estate_p2-4.xlsx

[Page 2] Loading https://www.yellowpages.com/queens-ny/real-estate?page=2
  Found 30 listings
  [ 1/30] Rb 9708 Realty                      [YP blocked, trying website] → (blocked, no website email found)
  [ 2/30] Prudential Action Real Estate       [YP blocked, trying website] → csg@reony.com
  [ 3/30] Award Homes Realty                  [YP blocked, trying website] → (blocked, no website email found)
  [ 4/30] Star global realty                  [YP blocked, trying website] → (blocked, no website email found)
  [ 5/30] Greenwald Realty                    [YP blocked, trying website] → (blocked, no website email found)
  [ 6/30] Achievers Real Estate & Advisors SE [YP blocked, trying website] → (blocked, no website email found)
  [ 7/30] Nationwide Homes                    [YP blocked, trying website] → (blocked, no website email found)
  [ 8/30] U